In [0]:
import pyspark
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType

# A list of folders containing sample datasets we can use
display(dbutils.fs.ls("/databricks-datasets/samples/"))

path,name,size,modificationTime
dbfs:/databricks-datasets/samples/adam/,adam/,0,0
dbfs:/databricks-datasets/samples/data/,data/,0,0
dbfs:/databricks-datasets/samples/docs/,docs/,0,0
dbfs:/databricks-datasets/samples/lending_club/,lending_club/,0,0
dbfs:/databricks-datasets/samples/newsgroups/,newsgroups/,0,0
dbfs:/databricks-datasets/samples/people/,people/,0,0
dbfs:/databricks-datasets/samples/population-vs-price/,population-vs-price/,0,0


In [0]:
# Loading in a sample table into the dataframe
df = spark.read.csv("/databricks-datasets/samples/population-vs-price/data_geo.csv", header=True)

In [0]:
# To view the first 20 rows of the df
df.show()
# OR we can add an integer into the parentheses to view a specific 
# number of rows
df.show(5)

+---------+-------------+----------+----------+------------------------+-----------------------+
|2014 rank|         City|     State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+----------+----------+------------------------+-----------------------+
|      101|   Birmingham|   Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|   Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|   Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|   Alabama|        AL|                  200481|                    129|
|       64|Anchorage[19]|    Alaska|        AK|                  301010|                   null|
|       78|     Chandler|   Arizona|        AZ|                  254276|                   null|
|       86|  Gilbert[20]|   Arizona|        AZ|                  239277|                   null|
|       88|     Glendale|   Ar

In [0]:
# Viewing the column names
df.columns

Out[18]: ['2014 rank',
 'City',
 'State',
 'State Code',
 '2014 Population estimate',
 '2015 median sales price']